In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [46]:
url_inicial = 'https://myb.ojs.inecol.mx/index.php/myb/issue/archive/4'
url_root = 'https://myb.ojs.inecol.mx/index.php/myb/issue/archive/4'
r=requests.get(url_inicial)

In [47]:
soup = BeautifulSoup(r.text, 'html.parser')

In [48]:
box = soup.find('div', class_='flex_container issues_list')
volumen=box.findAll('div',class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol

['https://myb.ojs.inecol.mx/index.php/myb/issue/view/224']

In [49]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='section_content') 
    volumen1=box1.findAll('article', class_='article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [50]:
vol2

['https://myb.ojs.inecol.mx/index.php/myb/article/view/1399']

In [51]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='section_content') 
    volumen1=box1.findAll('article', class_='article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [52]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://myb.ojs.inecol.mx/index.php/myb/issue/archive/4


In [53]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

1

In [54]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [55]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Madera y Bosques'
    a='VI. Biotecnología y Ciencias Agropecuarias'
    tem='Recursos Forestales'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page_title article-full-title').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='abstract').find('p').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='btn galley-link obj_galley_link pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [56]:
list_scraper=list_scraper[0:37]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0


In [57]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Madera y Bosques,VI. Biotecnología y Ciencias Agropecuarias,Recursos Forestales,Integración bosque industria: Una necesidad re...,None,None,https://myb.ojs.inecol.mx/index.php/myb/articl...,https://myb.ojs.inecol.mx/index.php/myb/articl...


In [58]:
df_catalogo.to_csv('Revista27.04.csv', index=False)